# Data

[150k Python Dataset](https://eth-sri.github.io/py150) from SRILAB.

# Code representation

A continuous vector embedding of each code fragment at method–level granularity as "document".[13] FastText, a variation of Word2Vec algorithm.
- Extracting Information from Source Code
    - simple tokenizer: extract all words from source code by removing non–alphanumeric tokens.=> indifferenciable
    - parser-based approach: traverse through the parse tree for each method, and extract information from the following syntactic categories. (Java-like)
        - method name
        - method invocation
        - Enums
        - String literals
        - comments
        - <strike>variable name</strike>
- Building Vector Representations
    - <strike>simply average embeddings</strike>
    - Weighted average of all unique words in a document=> normalized tf-idf
- Retrieval
    - average the vector representations of constituent words to create a document embedding for the query sentence
    - a standard similarity search algorithm to find the document vectors with closest cosine distance. => FAISS 

# Model

Input: natural language queries <br>
Output: related code fragments retrieved directly from Github code corpus<br><br>

Map the Input into the same vector space as the codebase, and then calculate the vector distance of them in order to get the relevant result.

# Evaluation

Metric and choosing parameters of the model

- Metric: select subsets of words from the document as simulated queries and then see if it can retrive the document, and then evaluate by the percentage of the documents that are retrieve back at top1 and top10. 
    - random benchmark test
    - TF-IDF benchmark test => better performance
- Parameters:
    - embedding dimention=> 500
    - three ways of combining word embeddings to document embeddings=> the conclusion is tf-idf better
    - vector representation=> BM25 is better

# Implementation

In [1]:
import numpy as np
import pandas as pd
import string
import re

from gensim.models import FastText

## Data Processing

In [2]:
# load in processed file. A set of keywords for each document (source code function)
unpickled_df = pd.read_pickle("data/py100.pkl")
func_size=len(unpickled_df)
print("Total Number of Functions: {}".format(func_size))
unpickled_df.head()

Total Number of Functions: 1038


,data_id,function_name,docstring,func_call
0,1,__getattribute__,,__getattribute__
1,1,__setattr__,,"ref,__setattr__"
2,2,main,,"setup,closing,ZmqProxy,consume_in_thread,wait"
3,3,test_vpnservice_create,,"create_stubs,first,AndReturn,ReplayAll,vpnserv..."
4,3,test_vpnservices_get,,"create_stubs,AndReturn,ReplayAll,vpnservices_g..."


In [3]:
def parse_func_name(terms):
    
    # remove punctuations except for "_"
    ## reference: https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
    regex = re.compile('[%s]' % re.escape(string.punctuation.replace("_", "")))
    terms=regex.sub('', terms)
    
    #[TODO] camel case
    
    # [TODO] deal with __init__

    # lowercase
    terms=terms.lower()
    
    # snake case
    return [term for term in terms.split("_") if term!=""]
    

In [4]:
# for each function, combine all the keywords into a set
list_function_keywords=[]
for idx in range(len(unpickled_df)):
    keywords=[]
    
    func_name=parse_func_name(unpickled_df.iloc[idx]["function_name"].lower())
    keywords+=func_name
    
    #[TODO] only alphabenumeric characters
    #[TODO] camel case
    #[TODO] snake case
    
    if unpickled_df.iloc[idx]["docstring"]:
        docstring=unpickled_df.iloc[idx]["docstring"].lower().split()
        keywords+=docstring
    
    if unpickled_df.iloc[idx]["func_call"]:
        func_invoc=unpickled_df.iloc[idx]["func_call"].lower().split(",")
        keywords+=func_invoc
    
    list_function_keywords.append(set(keywords))

In [5]:
len(list_function_keywords)

1038

## Building Word Embeddings

In [6]:
# hyperparameters
vocab_size=200
window_size=5
min_count=1


# other parameters defined earlier
# func_size

In [7]:
# We employ the continuous skip–gram model with a window size of 5, 
# i.e. all pairs of words within distance 5 are considered nearby words.

#[TODO] tuning hyperparameters
model = FastText(size=vocab_size, window=window_size, min_count=min_count)  # instantiate
model.build_vocab(sentences=list_function_keywords)
model.train(sentences=list_function_keywords, total_examples=len(list_function_keywords), epochs=10)  # train

In [8]:
print(model)

FastText(vocab=2826, size=200, alpha=0.025)


In [9]:
# saving a model trained via Gensim's fastText implementation
# 2019/03/05: the model might be too big. Saving word vector only.
# model.save('saved_model_gensim')

In [10]:
trained_ft_vectors = model.wv
# save vectors to file if you want to use them later
trained_ft_vectors.save_word2vec_format('embeddings.txt', binary=False)

In [11]:
# Test
trained_ft_vectors.most_similar("button", topn=10)

[('buttonpresssignal', 0.9999946355819702),
 ('menubutton', 0.9999945163726807),
 ('__menubutton', 0.9999942183494568),
 ('buttonpress', 0.9999940991401672),
 ('buttonreleasesignal', 0.9999937415122986),
 ('__selectionchangedsignal', 0.9999933242797852),
 ('__addbuttonclicked', 0.999993085861206),
 ('selectionchangedsignal', 0.9999930262565613),
 ('__selectionchanged', 0.9999929666519165),
 ('addbuttonclicked', 0.9999929666519165)]

## Building Document Embeddings

1. Average over all the words;
2. Average over the unique words in each document;
3. [x] Weighted average of all unique words in a document

In [12]:
trained_ft_vectors["ping"]

array([-0.42494965,  0.09168912,  0.2007297 ,  0.23383524, -0.27776867,
       -0.1292018 , -0.3475728 ,  0.03782819,  0.29607064,  0.11542961,
       -0.20680366,  0.15838797,  0.1562798 , -0.27328148,  0.23053621,
        0.04101355,  0.23876522,  0.33359843, -0.10300179, -0.19574092,
        0.47251424, -0.30474472,  0.02372519,  0.17662196,  0.12936169,
       -0.25550768, -0.32257852,  0.14303087,  0.18635245,  0.10497522,
       -0.49918306, -0.19945878, -0.20125861, -0.21386047,  0.10984774,
       -0.08012322, -0.2588905 , -0.03946825,  0.04088959,  0.11942113,
       -0.28353572, -0.07734952, -0.40091765,  0.03058645,  0.19201984,
        0.22799787, -0.02762045,  0.13490786,  0.14143841,  0.08460612,
       -0.02339482,  0.02143422, -0.09180233, -0.05974629,  0.2278142 ,
        0.44051978,  0.25641173,  0.27906194, -0.07556982,  0.08890175,
       -0.10538024, -0.48284948,  0.13223481, -0.0586512 ,  0.17563498,
       -0.25788468,  0.10269392, -0.4170083 ,  0.07758314,  0.10

In [13]:
document_embeddings=np.zeros((func_size, vocab_size))
for idx, doc in enumerate(list_function_keywords):
    doc_vec_sum=np.zeros(vocab_size)
    for term in doc:
        doc_vec_sum+=trained_ft_vectors[term]
    # [TODO]
    document_embeddings[idx]=doc_vec_sum

In [14]:
document_embeddings[0]

array([-0.68765068,  0.14810513,  0.32426845,  0.3791457 , -0.45057426,
       -0.21252298, -0.56160876,  0.06236718,  0.47689047,  0.18688078,
       -0.33587725,  0.25544155,  0.25432747, -0.44041638,  0.37236835,
        0.06855008,  0.38587447,  0.53741966, -0.16315433, -0.31615217,
        0.76546419, -0.49198873,  0.04245611,  0.28367878,  0.20849012,
       -0.41151454, -0.52255523,  0.23242894,  0.29964996,  0.17130568,
       -0.80605513, -0.32197484, -0.32747605, -0.34827352,  0.17611656,
       -0.13026425, -0.42065385, -0.06449596,  0.06474601,  0.19491098,
       -0.46017744, -0.12225904, -0.6469785 ,  0.05170663,  0.30724122,
        0.37122571, -0.04252634,  0.218674  ,  0.22875033,  0.13581955,
       -0.03804267,  0.03394479, -0.14949341, -0.09481506,  0.36656499,
        0.71355459,  0.4134258 ,  0.45066193, -0.1238381 ,  0.14311814,
       -0.17100363, -0.78071624,  0.21473414, -0.09691828,  0.28427614,
       -0.41771814,  0.16506787, -0.67654321,  0.12585035,  0.16

In [15]:
print("{} documents with {} dimentions".format(document_embeddings.shape[0], document_embeddings.shape[1]))

1038 documents with 200 dimentions


## Evaluate Model

# Notes

# Questions
The paper mentions 2 evaluation approach: 1 uses Github only, the other one uses both GitHub and StackOverflow. I'm guessing the former one is for tuning in the development stage; while the later is the final evaluation for the completed system (NCS).